In [4]:
import torch
import numpy as np
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

D:\cs\Anaconda\envs\deeplearning_common\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'D:\cs\Anaconda\envs\deeplearning_common\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [16]:
batch_size = 10
# transformer =transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize(mean = (0.5, 0.5, 0.5),std = (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.MNIST('./data',train=True,download=True,transform=transforms.ToTensor())
testset = torchvision.datasets.MNIST('./data',train=False,download=True,transform=transforms.ToTensor())
train_loader = DataLoader(trainset,batch_size=batch_size,shuffle=True,num_workers=2)
test_loader = DataLoader(testset,batch_size=batch_size,shuffle=True,num_workers=2)

In [35]:
print(type(trainset),'\n',type(testset),'\n',type(train_loader),'\n',type(test_loader))
train_iter = iter(train_loader)
print(type(train_iter))
train_x,train_y = train_iter.next()
print(type(train_x),type(train_y))
print(train_x.shape,train_y.shape)
train_data = trainset.data.numpy()
train_label = trainset.targets.numpy()
test_data = testset.data.numpy()
test_label = trainset.targets.numpy()
train_data = train_data.reshape(train_data.shape[0],-1)
test_data = test_data.reshape(test_data.shape[0],-1)
print(f'train_x{train_data.shape},train_y{train_label.shape},test_x{test_data.shape},test_y{test_label.shape}')
print('before ',train_data.mean())
train_data = train_data.astype(np.float32)
train_data = np.multiply(train_data, 1.0 / 255.0)
print('after ',train_data.mean())


<class 'torchvision.datasets.mnist.MNIST'> 
 <class 'torchvision.datasets.mnist.MNIST'> 
 <class 'torch.utils.data.dataloader.DataLoader'> 
 <class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.utils.data.dataloader._MultiProcessingDataLoaderIter'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([10, 1, 28, 28]) torch.Size([10])
train_x(60000, 784),train_y(60000,),test_x(10000, 784),test_y(60000,)
before  33.318421449829934
after  0.13066062


In [42]:
a = np.array([-0.14579831,  0.0507718 , -0.0928348 , -0.15952584,  0.03019532])
b = np.array([-0.14579831,  0.0507718 , -0.0928348 , -0.15952584,  0.03019532])
import torch.nn.functional as F
dist = np.linalg.norm(a-b)
print(dist)

0.0


In [5]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()

In [3]:
from cacheout import Cache
cache = Cache()

In [23]:
cache.set(1,'a')
assert cache.get(1) == 'a'
assert cache.get(2) is None
print(cache.get(1),cache.get(2))

a None


In [50]:
import copy
import pickle
import hashlib
net2 =copy.deepcopy(net)
def state_equal(sd1,sd2):
    for key in sd1:
            # print(key)
            if not sd1[key].equal(sd2[key]):
                return False
    return True
def net_equal(net1,net2):
    sd1 = net1.state_dict()
    sd2 = net2.state_dict()
    return state_equal(sd1,sd2)
print(net_equal(net,net2))
bytes_model2 = pickle.dumps(net2.state_dict())
bytes_model1 = pickle.dumps(net.state_dict())
print(bytes_model1[:10],bytes_model2[:10])

True
b'\x80\x04\x95!2\x00\x00\x00\x00\x00' b'\x80\x04\x95!2\x00\x00\x00\x00\x00'


In [82]:
import hashlib
import time
bytes_model1=pickle.dumps(net.state_dict)
# bytes_model2=pickle.dumps(net.state_dict)
# print(bytes_model1[:100],bytes_model2[:100])
# print(bytes_model1 == bytes_model2)
def get_hash(bytes_model):
    hash = hashlib.md5()
    print(id(hash))
    hash.update(bytes_model)
    # hash.update(b'haiwu')
    hash.update(bytes(str(time.time()),encoding="utf-8"))
    hex = hash.hexdigest()
    return hex
hex1 = get_hash(bytes_model1)
print(type(hex1))
cache.add(hex1,bytes_model1)

2322760581008
<class 'str'>


In [67]:
bytes_model1 = cache.get(hex1)
hex2 = get_hash(bytes_model1)
print(hex2,hex1)
print(len(cache))

2322760311440
21c4b39662c4f9e792b906370745059f 21c4b39662c4f9e792b906370745059f
2


In [93]:
import math
n_p = 5
n_v = 3
def sigmoid(x):
    return 1/(1+math.exp(-x))
for i in range(6):
    print(sigmoid(i-n_v),i)
print(0.88/0.5,5/3)

0.04742587317756678 0
0.11920292202211755 1
0.2689414213699951 2
0.5 3
0.7310585786300049 4
0.8807970779778823 5
1.76 1.6666666666666667
